# 지역별 해양기후에 영향을 주는 요소 정리하기



## 컬럼설명
##### id = 지점번호
##### station_na = 지역명칭
##### sensor_typ = 부이,파고부이,등표 3개로 나뉘어짐 --> 순서대로['Buoy', 'Costal_wave_buoy', 'Light_house_aws']
##### latitude = 위도
##### logitude = 경도
##### region = 한반도 서해,남해,동해를 1구역~5구역까지 5개구역으로 구역으로 나눔 
##### region_sub =대륙을 중심으로 가까운바다,중간바다,먼바다를 기준으로 순서대로 1,2,3으로 나눔
##### yaer = 연도
##### month = 월
##### 이하 생략

In [226]:
import pandas as pd
import os

#엑셀 시트별로 파일 불러오는 함수
def excel(sheet_name):
    data_folder = "data"
    # 각 연도별 폴더 리스트
    years = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
    # 모든 데이터를 저장할 빈 리스트
    all_data = []
    # 각 연도별 폴더를 순회하면서 파일을 읽어옴
    for year in years:
        # 해당 연도의 폴더 경로
        year_folder = os.path.join(data_folder, year)
        
        # 해당 연도 폴더 안의 모든 파일을 읽어옴
        for file in os.listdir(year_folder):
            # 파일 경로
            file_path = os.path.join(year_folder, file)
            # 파일이 xls나 xlsx인 경우에만 읽어옴
            if file.endswith(".xls") or file.endswith(".xlsx"):
                try:
                    # 파일을 DataFrame으로 읽어옴
                    data = pd.read_excel(file_path, header=1, sheet_name=sheet_name)
                    all_data.append(data)
                    # -------> 파일안에 연도와 월 컬림이 없어 미리 생성해둘 예정
                    file_name = os.path.splitext(file)[0]  
                    year_month = file_name.split('_')[1]    
                    year_str = year_month[:4]
                    month_str = year_month[4:]
                    
                    # 'year'와 'month' 열 추가
                    data['year'] = int(year_str)
                    data['month'] = int(month_str)
                except ValueError:
                    file_name = os.path.splitext(file)[0]  
                    year_month = file_name.split('_')[1]    
                    year_str = year_month[:4]
                    month_str = year_month[4:]
                    # 지정된 시트가 존재하지 않는 경우
                    print(f"파일 '{file_path}'에 시트 '{sheet_name}'가 존재하지 않습니다.")
                    # 다른 시트를 기본값으로 사용하여 파일을 다시 읽어옴
                    data = pd.read_excel(file_path, header=1)
                    all_data.append(data)
                
    # 모든 데이터를 하나의 DataFrame으로 결합
    combined_data = pd.concat(all_data, ignore_index=True)
    return combined_data


In [227]:
# 1번시트 불러오기 
# 2분 20초 소요
combined_data = excel('월요약자료-부이')

In [228]:
# 1번시트 엑셀 파일로 저장
combined_data.to_excel("./data/combined_data.xlsx", index=False)

In [229]:
#2번시트 불러오기
# 2분 20초 소요
combined_data2 = excel('월요약자료-등표')

In [230]:
# 2번시트 엑셀 파일로 저장
combined_data2.to_excel("./data/combined_data2.xlsx", index=False)

In [231]:
# 2015년 1월부터 3월까지는 파고부이에 대한 월요약자료가 없음 그래서 제외하고 진행
combined_data3 = excel('월요약자료-파고부이')

In [232]:
#3번시트 엑셀 파일로 저장
combined_data3.to_excel("./data/combined_data3.xlsx", index=False)

In [235]:
#미리 만들어둔 데이터 불러오기 
df = pd.read_csv('./data/station.csv')

In [236]:
df.head()

,id,station_na,sensor_typ,latitude,longitude,region,region_sub,station_no
0,22191,서해170,Buoy,36.133333,124.056944,1,3,22191
1,22192,서해206,Buoy,34.000000,123.262500,2,3,22192
2,22299,서해190,Buoy,35.000000,124.133333,2,3,22299
3,22300,남해239,Buoy,32.833333,124.733333,3,3,22300
4,22301,남해465,Buoy,31.666667,126.400000,3,3,22301


In [237]:
#id랑 station_no랑 중복되어서 station_no 컬럼 삭제
df = df.drop(columns='station_no',axis=1,)

In [238]:
df

,id,station_na,sensor_typ,latitude,longitude,region,region_sub
0,22191,서해170,Buoy,36.133333,124.056944,1,3
1,22192,서해206,Buoy,34.000000,123.262500,2,3
2,22299,서해190,Buoy,35.000000,124.133333,2,3
3,22300,남해239,Buoy,32.833333,124.733333,3,3
4,22301,남해465,Buoy,31.666667,126.400000,3,3
...,...,...,...,...,...,...,...
106,22302,동해78,Buoy,36.966667,130.516667,5,2
107,22442,혈암,Costal_wave_buoy,37.541389,130.854167,5,2
108,22443,구암,Costal_wave_buoy,37.478889,130.804444,5,2
109,22464,울릉읍,Costal_wave_buoy,37.471389,130.900278,5,2


In [271]:
# 위에서 저장한 엑셀파일 불러오기
com_data = pd.read_excel('./data/combined_data.xlsx')
com_data2 = pd.read_excel('./data/combined_data2.xlsx')
com_data3 = pd.read_excel('./data/combined_data3.xlsx')

In [272]:
#지점번호에 NAN값이 있으면 전체행 삭제
com_data = com_data.dropna(subset=['지점번호'])
com_data2 = com_data2.dropna(subset=['지점번호'])
com_data3 = com_data3.dropna(subset=['지점번호'])

In [275]:
# 지점번호 float --> int로 변경
com_data['지점번호'] = com_data['지점번호'].astype(int)
com_data2['지점번호'] = com_data2['지점번호'].astype(int)
com_data3['지점번호'] = com_data3['지점번호'].astype(int)


In [276]:
com_data.head()

,지점번호,지점명,유의파고,최대파고,Unnamed: 4,Unnamed: 5,Unnamed: 6,평균(sec),최대(sec),나타난날,...,나타난날.2,일평균최저(℃),나타난날.3,평균(℃).1,일평균최고(℃).1,나타난날.4,일평균최저(℃).1,나타난날.5,year,month
2,21229,울릉도,1.2,6.7,03,9,3,5.6,9.1,21.0,...,27.0,5.9,7.0,11.4,14.0,30.0,10.6,7.0,2015,4
3,22101,덕적도,0.6,2.9,16,NaN,NaN,3.8,12.8,5.0,...,29.0,6.0,8.0,7.0,8.7,30.0,5.5,2.0,2015,4
4,22102,칠발도,1.1,6.2,07,9,3,4.5,10.7,5.0,...,28.0,6.6,7.0,10.6,13.4,27.0,8.7,2.0,2015,4
5,22103,거문도,1.1,4.5,04,12,NaN,6.2,10.7,7.0,...,27.0,10.0,14.0,12.9,14.0,27.0,11.7,2.0,2015,4
6,22104,거제도,1.2,6.0,13,11,2,5.7,9.1,22.0,...,28.0,9.8,8.0,13.8,14.8,28.0,13.2,6.0,2015,4


In [277]:
print(com_data[com_data['year'].isna()])
print(com_data2[com_data2['year'].isna()])
print(com_data3[com_data3['year'].isna()])

Empty DataFrame
Columns: [지점번호, 지점명, 유의파고, 최대파고, Unnamed: 4, Unnamed: 5, Unnamed: 6, 평균(sec), 최대(sec), 나타난날, 풍속(㎧), 일평균최대(㎧), 나타난날.1, Unnamed: 13, Unnamed: 14, 평균(℃), 일평균최고(℃), 나타난날.2, 일평균최저(℃), 나타난날.3, 평균(℃).1, 일평균최고(℃).1, 나타난날.4, 일평균최저(℃).1, 나타난날.5, year, month]
Index: []

[0 rows x 27 columns]
Empty DataFrame
Columns: [지점번호, 지점명, 유의파고, 최대파고, Unnamed: 4, Unnamed: 5, Unnamed: 6, 평균(sec), 최대(sec), 나타난날, 평균(m), 최대(m), 나타난날.1, Unnamed: 13, Unnamed: 14, 평균풍속(㎧), 최대순간, Unnamed: 17, Unnamed: 18, 평균(℃), 일최고(℃), 나타난날.2, 일최저(℃), 나타난날.3, 평균(℃).1, 일평균최고(℃), 나타난날.4, 일평균최저(℃), 나타난날.5, year, month]
Index: []

[0 rows x 31 columns]
Empty DataFrame
Columns: [지점번호, 지점명, 유의파고, 최대파고, Unnamed: 4, Unnamed: 5, Unnamed: 6, 평균(sec), 최대(sec), 나타난날, 평균(℃), 일평균최고(℃), 나타난날.1, 일평균최저(℃), 나타난날.2, year, month]
Index: []


In [278]:

# df와 com_data를 합칠 때 사용할 공통 열 기준으로 병합
merged_df = pd.merge(df, com_data[['지점번호','유의파고', '최대파고', '풍속(㎧)','year','month']], how='left', left_on='id', right_on='지점번호')
merged_df2 = pd.merge(df, com_data2[['지점번호','유의파고', '최대파고', '평균풍속(㎧)','year','month']], how='left', left_on='id', right_on='지점번호')
merged_df3 = pd.merge(df, com_data3[['지점번호','유의파고', '최대파고','year','month']], how='left', left_on='id', right_on='지점번호')



In [280]:
# 데이터프레임들을 리스트에 저장
merged_dfs = [merged_df, merged_df2, merged_df3]
# 데이터프레임들을 병합하여 하나의 데이터프레임으로 합침
merged_combined = pd.concat(merged_dfs, ignore_index=True)
# 중복값 제거
merged_combined_unique = merged_combined.drop_duplicates()



In [281]:
#year 결측치 확인
merged_combined_unique[merged_combined_unique['year'].isna()]

,id,station_na,sensor_typ,latitude,longitude,region,region_sub,지점번호,유의파고,최대파고,풍속(㎧),year,month,평균풍속(㎧)
197,22441,독도,Costal_wave_buoy,37.238611,131.871389,5,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331,955,서수도,Light_house_aws,37.325000,126.393333,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
332,956,가대암,Light_house_aws,36.770000,125.976667,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333,22444,신진도,Costal_wave_buoy,36.605000,126.126111,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,22445,삽시도,Costal_wave_buoy,36.371389,126.338333,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3160,21229,울릉도,Buoy,37.450000,131.100000,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3161,22105,동해,Buoy,37.480556,129.950000,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3162,22106,포항,Buoy,36.350000,129.783333,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3163,22190,울진,Buoy,36.906944,129.874444,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [283]:
#결측치 삭제
merged_combined_unique.dropna(subset = ['year'])

,id,station_na,sensor_typ,latitude,longitude,region,region_sub,지점번호,유의파고,최대파고,풍속(㎧),year,month,평균풍속(㎧)
0,22191,서해170,Buoy,36.133333,124.056944,1,3,22191.0,1.2,8.5,7.3,2019.0,12.0,NaN
1,22191,서해170,Buoy,36.133333,124.056944,1,3,22191.0,1.1,8.5,7.1,2020.0,1.0,NaN
2,22191,서해170,Buoy,36.133333,124.056944,1,3,22191.0,1.2,8.7,6.7,2020.0,2.0,NaN
3,22191,서해170,Buoy,36.133333,124.056944,1,3,22191.0,1.2,7.3,6.5,2020.0,3.0,NaN
4,22191,서해170,Buoy,36.133333,124.056944,1,3,22191.0,0.9,5.2,6.1,2020.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9407,22506,울릉서부,Costal_wave_buoy,37.433333,130.500000,5,2,22506.0,0.7,4.2,NaN,2023.0,8.0,NaN
9408,22506,울릉서부,Costal_wave_buoy,37.433333,130.500000,5,2,22506.0,0.7,4.9,NaN,2023.0,9.0,NaN
9409,22506,울릉서부,Costal_wave_buoy,37.433333,130.500000,5,2,22506.0,0.8,4.8,NaN,2023.0,10.0,NaN
9410,22506,울릉서부,Costal_wave_buoy,37.433333,130.500000,5,2,22506.0,NaN,NaN,NaN,2023.0,11.0,NaN


In [288]:
#year에 값이 없는 행은 전체다 삭제
merged_combined_unique = merged_combined_unique.dropna(subset=['year'])


In [301]:
# year,month컬럼 float --> int로 변경
merged_combined_unique[['year','month']] = merged_combined_unique[['year','month']].astype(int)


C:\Users\user\AppData\Local\Temp\ipykernel_31120\272889262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_combined_unique[['year','month']] = merged_combined_unique[['year','month']].astype(int)


In [302]:
merged_combined_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9177 entries, 0 to 9411
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          9177 non-null   int64  
 1   station_na  9177 non-null   object 
 2   sensor_typ  9177 non-null   object 
 3   latitude    9177 non-null   float64
 4   longitude   9177 non-null   float64
 5   region      9177 non-null   int64  
 6   region_sub  9177 non-null   int64  
 7   지점번호        9177 non-null   float64
 8   유의파고        7807 non-null   object 
 9   최대파고        7807 non-null   object 
 10  풍속(㎧)       1944 non-null   float64
 11  year        9177 non-null   int32  
 12  month       9177 non-null   int32  
 13  평균풍속(㎧)     831 non-null    float64
dtypes: float64(5), int32(2), int64(3), object(4)
memory usage: 1003.7+ KB


In [305]:
#병합파일 엑셀로 저장
merged_combined_unique.to_excel('./data/combined_unique.xlsx')